<a href="https://colab.research.google.com/github/CC-MNNIT/2018-19-Classes/blob/master/MachineLearning/2019_04_11_ML3_content/object_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2019 MNNIT Computer Club.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

> Reading someone else's code is more important than writing your own.

# Before we begin

In [0]:
# Install kaggle API
!pip install -q kaggle

1. Go to your kaggle account page
2. click ```Create New API Token```
3. A json file shall be downloaded to your system
4. Run the following cell, and upload that json file

**Please accept the competition rules before proceeding** since we will be using their dataset

[Competition rules : cats and dogs](https://www.kaggle.com/c/dogs-vs-cats/rules)

## A few driver functions

I have written a few functions, which can be used on colab to save your time. Since they solve a repetitve task.

In [0]:
def download_extract_colab(API_string, reupload_JSON=False, json_filename="/content/kaggle.json"):
    
    # TODO: 
    # improve ls output at the end, distinguish between files and directories, preferably linux ls -al like output
    # Add doc string
    # add more comments
    
    
    import os,shutil,sys  
            
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")) or reupload_JSON == True:
        
        from google.colab import files
        
        print("Please upload your kaggle API file...")
        files.upload()

        if not os.path.exists(os.path.expanduser("~/.kaggle/")):
            os.mkdir(os.path.expanduser("~/.kaggle/"))

        os.rename(json_filename,os.path.expanduser("~/.kaggle/kaggle.json"))
        os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"),600)
    
    
    
    if os.path.exists("/content/dataset"):
        shutil.rmtree("/content/dataset")
    os.mkdir("/content/dataset")
    

    sys.stdout.write("Downloading the dataset...")
    # download the dataset
    os.system(API_string+" -p /content/dataset/")
    sys.stdout.write("Done!\n")
    

    # extract the zip files into dataset folder
    sys.stdout.write("Extracting the dataset...")
    os.system("unzip '/content/dataset/*.zip' -d /content/dataset/; rm /content/dataset/*.zip")
    sys.stdout.write("Done!\n")
    
    print("===================")
    print("Dataset files downloaded at /content/dataset :")
    for file in os.listdir("/content/dataset"):
        print("\t/content/dataset/",file)
    print("===================")
    
    return "/content/dataset"

In [0]:
# the following function can be saved, since it will be extensively reused

def plot_loss_acc(history):
    
    import matplotlib.pyplot as plt
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
    plt.show()

# Let's begin

## Download Data

In [0]:
import os,shutil

In [0]:
original_dataset_dir = download_extract_colab("kaggle competitions download -c dogs-vs-cats")

## Create a proper folder structure to store the data

Since it's a binary object classification problem. 
- we can exploit the [flow_from_directory method](https://keras.io/preprocessing/image/#flow_from_directory) of keras.
- The method requires that the data is organised as follows:
    - data
        - dogs
        - cats
- i.e the data directory should contain one subdirectory per class.
- So, all the dog images under a directory named dogs (and similarly for cats) we can save ourselves the hassle of manually creating batches.

In [0]:
# audit how the dataset looks like and read the documentation about the dataset from kaggle

os.listdir(os.path.join(original_dataset_dir,"train"))
# os.listdir(os.path.join(original_dataset_dir,"test1"))

In [0]:
# directories where we store data after preprocessing (if any)
base_dir = "/content/project"
os.mkdir(base_dir)




# training directories
train_dir = os.path.join(base_dir,"train")
os.mkdir(train_dir)

train_dogs_dir = os.path.join(train_dir,"dogs")
os.mkdir(train_dogs_dir)

train_cats_dir = os.path.join(train_dir,"cats")
os.mkdir(train_cats_dir)




# validation directories
validation_dir = os.path.join(base_dir,"validation")
os.mkdir(validation_dir)

validation_dogs_dir = os.path.join(validation_dir,"dogs")
os.mkdir(validation_dogs_dir)

validation_cats_dir = os.path.join(validation_dir,"cats")
os.mkdir(validation_cats_dir)




# testing directories
test_dir = os.path.join(base_dir,"test")
os.mkdir(test_dir)

test_dogs_dir = os.path.join(test_dir,"dogs")
os.mkdir(test_dogs_dir)

test_cats_dir = os.path.join(test_dir,"cats")
os.mkdir(test_cats_dir)

## Populate Empty Directories

In [0]:
# populate the empty cat directories


fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for cat in fnames:
    src = os.path.join(original_dataset_dir,"train",cat)
    dst = os.path.join(train_cats_dir,cat)
    shutil.copyfile(src,dst)    
    

fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for cat in fnames:
    src = os.path.join(original_dataset_dir,"train",cat)
    dst = os.path.join(validation_cats_dir,cat)
    shutil.copyfile(src,dst)
    

fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for cat in fnames:
    src = os.path.join(original_dataset_dir,"train",cat)
    dst = os.path.join(test_cats_dir,cat)
    shutil.copyfile(src,dst)
    

# populate the empty dog directories


fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for dog in fnames:
    src = os.path.join(original_dataset_dir,"train",dog)
    dst = os.path.join(train_dogs_dir,dog)
    shutil.copyfile(src,dst)    

fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for dog in fnames:
    src = os.path.join(original_dataset_dir,"train",dog)
    dst = os.path.join(validation_dogs_dir,dog)
    shutil.copyfile(src,dst)    
    

fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for dog in fnames:
    src = os.path.join(original_dataset_dir,"train",dog)
    dst = os.path.join(test_dogs_dir,dog)
    shutil.copyfile(src,dst)    

    
# len(os.listdir(test_dogs_dir))    

## Build a keras model

### FIRST : define datagenerators

Since we are dealing with image data, it would be preferable if we could create data generators which could automatically create batches of images (and additionally perform data augmentation) and supply that to the fit_generator method of a keras model.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

validation_gen = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

### From Scratch (baseline model)

In [0]:
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model

from keras import backend as K

In [0]:
# to reset the tf graph
K.clear_session()

In [0]:
model = Sequential()

model.add( layers.Conv2D(32,(3,3),activation='relu', input_shape=(150,150,3)) )
model.add( layers.MaxPooling2D((2,2)))

model.add( layers.Conv2D(64,(3,3),activation='relu' ))
model.add( layers.MaxPooling2D((2,2)))

model.add( layers.Conv2D(128,(3,3),activation='relu' ))
model.add( layers.MaxPooling2D((2,2)))
          
model.add( layers.Conv2D(128,(3,3),activation='relu' ))
model.add( layers.MaxPooling2D((2,2)))
          
# model.add( layers.Conv2D(224,(3,3),activation='relu' ))
# model.add( layers.MaxPooling2D((2,2)))

model.add( layers.Flatten() )
model.add(layers.Dropout(0.5))
model.add( layers.Dense(512,activation='relu'))
# sigmoid since our classification problem is binary -> 0 - dogs, 1 - cats
model.add( layers.Dense(1,activation='sigmoid'))

In [0]:
model.summary()

In [0]:
from IPython.display import Image
plot_model(model,show_shapes=True,show_layer_names=True,to_file="baseline_model.png",rankdir='TB')
Image(retina=True, filename='baseline_model.png')

In [0]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [0]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_gen,
    validation_steps=50
    )

In [0]:
model.save("cats_dog_1.h5")

In [0]:
plot_loss_acc(history)

### Using Transfer learning (very practical)


- Use a pre-trained model (a CNN). 
- Since CNNs learn low level patterns(eg. edges, curves etc) in their initial layers, we can freeze the initial layers and allow the model only to adjust the weights of the later layers (which learn high level features such as a cat or a dog).

---
To understand transfer learning : view a CNN model as
>>>>>> Convolution base + Classifier (= Last Dense layer/Linear Model)
---

There are two ways transfer learning can be used :

1. Feature Extraction :
    - Freeze all but the last layer (Dense) :
        - view the last dense layer as a perceptron model mimicing a linear model
        - it can be replaced via any other classifier (eg. a decision tree)
    - Use the convolution base as it is (since it has already learnt lower level representations on a bigger dataset)
        - the output of the conv base are basically just extracted features.
        - The weights of this convolution base are LOCKED. So that they don't change during the training process
    - Train this model on your dataset. Only the weights of the last layer are allowed to change, which learn the higher level features of the problem.
    - Test the model
2. Fine Tuning
    - Freeze all but the last FEW layers (Dense and some convolutional layers)
    

#### Feature Extraction

In [0]:
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model

from keras.applications.vgg16 import VGG16

Use VGG16 with imagenet weights as the convolution base

In [0]:
# include_top=False -> do not include the Dense layer of the VGG16 network
conv_base = VGG16(include_top=False,
                  weights="imagenet",
                  input_shape=(150,150,3))

# Since all the layers are yet not freezed
# their weights will be changed upon training
# so let's freeze the conv base
conv_base.trainable = False
conv_base.summary()

Create a CNN model
- Use the locked VGG16 CNN without it's dense layer as the conv base.
- Add your own Dense Layer (Classifier) at the end

In [0]:
t_model = Sequential()

t_model.add( conv_base )
t_model.add( layers.Flatten() )
t_model.add( layers.Dense(256,activation='relu') )
t_model.add( layers.Dense(1,activation='sigmoid') )

In [0]:
t_model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['acc'])

In [0]:
t_model.summary()

In [0]:
history = t_model.fit_generator(train_gen,
                                steps_per_epoch=100,
                                epochs=30,
                                validation_data=validation_gen,
                                validation_steps=50
                               )

In [0]:
plot_loss_acc(history)

In [0]:
t_model.save("transfer_learned_model_cats_dogs.h5")

#### Visualise Results

In [0]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [0]:
# draw a batch of test images from the test generator
imgs,labels = next(test_gen)

# make predictions on this batch
predictions = t_model.predict_classes(imgs,batch_size=test_gen.batch_size)

In [0]:
# convert the integer(0/1) class labels to original strings -> dog, cat

rev_map = { v:k[:-1] for k,v in test_gen.class_indices.items() }
predictions = [rev_map[i[0]] for i in predictions]
labels = [rev_map[i] for i in labels.astype('int')]

In [0]:
row = col = 4

fig,all_ax = plt.subplots(row,col,figsize=(20,20))

for i in range(len(all_ax)):
    for j,ax in enumerate(all_ax[i]):
        
        curr_idx = len(all_ax)*i+j
        
        ax.imshow(imgs[curr_idx])
        ax.set_title( "Prediction:"+predictions[curr_idx]+" | Actual:"+labels[curr_idx])
        ax.axis('off')
        ax.grid(False)
    
plt.show()
    

In [0]:
####### Redundant code

# # download the dataset
# !kaggle competitions download -c dogs-vs-cats -p dataset/

# # extract the zip files into dataset folder
# !unzip 'dataset/*.zip' -d dataset/
# !rm dataset/*.zip

---

Authored By [Dipunj Gupta](https://github.com/dipunj) | Report errors/typos as github issues.

---